###Context
You are a helpful assistant. You will be given multiple choice answers for each question. Your final answer should be formatted with "the correct answer is [your choice]."
###Prompts
Explain this question in at least 50 words. Then solve for the answer.
Explain this problem in at least 50 words. Then solve for the answer.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets
#!pip install transformers
#!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00


In [3]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [4]:
from datasets import load_dataset
import json
dataset = load_dataset("dmayhem93/agieval-sat-math")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/220 [00:00<?, ? examples/s]

In [5]:
question = dataset['test'][0]['query'].split('A:')[0]

In [6]:
true_answer = str(dataset['test'][0]['gold'])
if true_answer == "[0]":
  true_answer = "A"
elif true_answer == "[1]":
  true_answer = "B"
elif true_answer == "[2]":
  true_answer = "C"
elif true_answer == "[3]":
  true_answer = "D"
if our_answer == true_answer:
  print("True")

NameError: name 'our_answer' is not defined

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

In [ ]:
!pip install flash-attn

In [ ]:
chain_of_thought_aqua = """Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there
will be 21 trees. How many trees did the grove workers plant today?
A)6 B)7 C)2 D)4
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted. So,
they must have planted 21 - 15 = 6 trees. The correct answer is A.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A)2 B)4 C)5 D)1
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The correct answer is C.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A)40 B)39 C)20 D)25
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74 chocolates.
35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The correct answer is B.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did
Jason give to Denny?
A)18 B)10 C)6 D)8
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The correct answer is D.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he
have now?
A)10 B)3 C)11 D)9
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so in
total he has 7 + 2 = 9 toys. The correct answer is D."""

In [ ]:
for i,batch in enumerate(dataset["test"]):
  print(i)
  print(batch['question'])
  options = ''
  for o in batch['options']:
    options = options + " " + o
  print(options)
  print(batch["correct"])
  break

NameError: ignored

In [7]:
from torch.utils.data import DataLoader
import re

def extract_last_answer(full_text):
  try:
    segments = full_text.split("A: ")
    last_answer = segments[-1].strip()
    last_answer = last_answer.split("Question 1:")[0].strip()
    return last_answer

  except ValueError as e:
    return None

def extract_output_numerical_answer(text, key_phrase="The answer is"): # You may need to create variations of this function to extract your intended numerical answer, depending on the output of your prompt.
    if text is None:
        return None

    try:
        # Find the position where 'The answer is' starts in the text
        start_pos = text.index(key_phrase) + len(key_phrase)

        # Extract the text that comes after 'The answer is'
        answer_text = text[start_pos:].strip()

        # Use a regular expression to find a floating point or integer number
        matches = re.findall(r'[-+]?\d*\.\d+|\d+', answer_text)

        if matches:
            return float(matches[0])
        else:
            return None

    except ValueError:
        return None

In [ ]:
from torch.utils.data import DataLoader
import re


def extract_output_numerical_answer(text, key_phrase="The answer is"): # You may need to create variations of this function to extract your intended numerical answer, depending on the output of your prompt.


def extract_true_numerical_answer(text):


In [ ]:
key = "" #enter your key here


#GPT 3.5 Turbo

In [9]:
import openai
from openai import OpenAI
import re
import time

client = OpenAI(api_key=key)

correct = 0


s,e = 0,len(dataset["test"]) - 1

with (open("/content/drive/MyDrive/Full_SAT_Log/GPT3.5_Turbo/new_QAP25_Result.txt", "w") as f,
    open("/content/drive/MyDrive/Full_SAT_Log/GPT3.5_Turbo/new_QAP25_Answer.txt", "w") as d):

  f.write("new QAP25.: \n")
  d.write("new QAP25.: \n")
  for i, batch in enumerate(dataset["test"]): # Change debug_dataloader to train_dataloader or dev_dataloader when necessary
    if i < s:
      continue
    if(i > e):
      break
    question = batch['query'].split('A:')[0]
    #prompt = f"{question} Take a deep breath and work on this problem step-by-step."
    prompt = f"{question} Explain this problem to me in at least 25 words. Then solve for the answer."
    #prompt = f"{question}"
    #prompt = f"{chain_of_thought_aqua}\n{question}"
    #prompt = f"{question} Let's first understand the problem, extract relevant variables and their corresponding numerals, " \
    #         "and devise a complete plan. Then, let's carry out the plan, calculate intermediate variables " \
    #         "(pay attention to correct numerical calculation and commonsense), " \
    #         "solve the problem step by step, and show the answer."
    true_answer = str(batch['gold'])
    if true_answer == "[0]":
      true_answer = "A"
    elif true_answer == "[1]":
      true_answer = "B"
    elif true_answer == "[2]":
      true_answer = "C"
    elif true_answer == "[3]":
      true_answer = "D"
    response = client.chat.completions.create(
    model= "gpt-3.5-turbo", # "gpt-4-1106-preview"
    temperature = 0,
    seed=42,
    messages=[
      {"role": "system", "content": "You are a helpful assistant. You will be given multiple choice answers for each question. Your final answer should be formatted with 'the correct answer is [your choice].'"},
      {"role": "user", "content": prompt}
             ]
    )
    answer_line = response.choices[0].message.content
    our_answer = re.findall("the correct answer is \(*([A|B|C|D])\)*", answer_line,re.IGNORECASE)
    if our_answer == []:
      our_answer = "[]"
    else:
      our_answer = our_answer[0]
    d.write(f"Question[{i}]:\n {question}\n")
    d.write(f"Correct answer:\n {true_answer}\n")
    d.write(f"Full answer:\n {answer_line}\n")
    d.write(f"Our answer:\n {our_answer}\n")
    d.write("-"*100+"\n")
    if our_answer and our_answer == true_answer:
      correct += 1
      f.write(f"{i} Correct \n")
      print(f"{i} Correct")
    else:
      f.write(f"{i} Incorrect \n")
      print(f"{i} Incorrect")
    d.flush()
    f.flush()
    #if (i + 1) % 3 == 0:
    #  time.sleep(60)
  score = (correct/(i))*100
  print(score)
  f.write(f"Score: {score}")

0 Correct
1 Correct
2 Correct
3 Correct
4 Incorrect
5 Correct
6 Incorrect
7 Incorrect
8 Incorrect
9 Incorrect
10 Incorrect
11 Incorrect
12 Correct
13 Incorrect
14 Incorrect
15 Incorrect
16 Incorrect
17 Correct
18 Correct
19 Correct
20 Incorrect
21 Incorrect
22 Incorrect
23 Incorrect
24 Correct
25 Correct
26 Correct
27 Incorrect
28 Incorrect
29 Incorrect
30 Incorrect
31 Incorrect
32 Correct
33 Incorrect
34 Correct
35 Incorrect
36 Correct
37 Correct
38 Incorrect
39 Incorrect
40 Incorrect
41 Incorrect
42 Correct
43 Correct
44 Incorrect
45 Incorrect
46 Correct
47 Incorrect
48 Incorrect
49 Incorrect
50 Correct
51 Correct
52 Incorrect
53 Incorrect
54 Incorrect
55 Incorrect
56 Incorrect
57 Incorrect
58 Incorrect
59 Incorrect
60 Incorrect
61 Correct
62 Incorrect
63 Incorrect
64 Incorrect
65 Incorrect
66 Incorrect
67 Incorrect
68 Correct
69 Incorrect
70 Incorrect
71 Incorrect
72 Incorrect
73 Correct
74 Incorrect
75 Incorrect
76 Correct
77 Correct
78 Incorrect
79 Correct
80 Correct
81 Correct
82

#Phi Model

In [ ]:
def my_test():
  with (open("/content/drive/MyDrive/Aqua_Log/Phi/TABDPhi_Result.txt", "w") as f,
    open("/content/drive/MyDrive/Aqua_Log/Phi/TABDPhi_Answer.txt", "w") as d):

    f.write("Take a deep breath and work on this problem step-by-step.: \n")
    d.write("Take a deep breath and work on this problem step-by-step.: \n")
    s, e = 0, 99
    correct = 0
    for i, batch in enumerate(train_dataloader): # Change debug_dataloader to train_dataloader or dev_dataloader when necessary
      if i < s:
        continue
      if(i > e):
        break
      question = batch['question']
      options = ''
      for o in batch['options']:
        options = options + " " + o
      #prompt = f"{question} \n {options} \n Take a deep breath and work on this problem step-by-step."
      prompt = f"{question} \n {options} \n Explain this problem to me in at least 200 words. Then solve for the answer."
      #prompt = f"{question}\n{options}\n"
      #prompt = f"{question} Explain this problem to me in at least 200 words. Then solve for the answer."
      #prompt = f"{question} Take a deep breath and work on this problem step-by-step."
      #prompt = question
      #prompt = question_analysis_prompt_few_shot + "\n" + "Q: " + question
      #prompt = QAP_COT_Combination + "\n" + "Q: " + question
      true_answer = batch['correct']
      numerical_true_answer = extract_true_numerical_answer(true_answer)
      inputs = tokenizer(prompt, return_tensors="pt")
      inputs = inputs.to("cuda")
      outputs = model.generate(**inputs, max_new_tokens=350, do_sample=False) # You may need to modify max_new_tokens

      answer_line = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
      our_answer = re.findall("the correct answer is ([A|B|C|D|E])", answer_line,re.IGNORECASE)
      if our_answer == []:
        our_answer = "[]"
      else:
        our_answer = our_answer[0]
      d.write(f"Question[{i}]:\n {question}\n")
      d.write(f"Correct answer:\n {true_answer}\n")
      d.write(f"Full answer:\n {answer_line}\n")
      d.write(f"Our answer:\n {our_answer}\n")
      d.write("-"*100+"\n")
      if our_answer and our_answer == true_answer:
        correct += 1
        f.write(f"{i} Correct \n")
        print(f"{i} Correct")
      else:
        f.write(f"{i} Incorrect \n")
        print(f"{i} Incorrect")
      d.flush()
      f.flush()
      if (i + 1) % 3 == 0:
        time.sleep(60)
    score = (correct/(i))*100
    print(score)
    f.write(f"Score: {score}")
my_test()

FileNotFoundError: ignored

In [ ]:
from matplotlib import pyplot as plt


x = ["Baseline", "QAP50", "QAP100","QAP150", "TADB"]

y = [52, 54, 56, 61, 44]
plt.yticks([i for i in range(0,100,5)])
bar_container = plt.bar(x,y)
plt.bar_label(bar_container)
plt.show()

In [ ]:
import re

with (open("/content/drive/MyDrive/GSM8K_Log/GPT3.5_Turbo/QAP200_Answer.txt", "r") as inp):

  content = inp.read()
  txt = re.findall("Full answer:(.*?)Numeric answer:",content,re.DOTALL)
print(len(txt))
count = []
for t in txt:
  count.append(len(t.split()))

#print(count)
import statistics
statistics.mean(count)

In [ ]:
from matplotlib import pyplot as plt
x = [i for i in range(len(count))]
plt.bar(x,count)